#### Import Modules

In [1]:
import re
import io
import json
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import nltk
import functools
import tokenizations
from nltk.tokenize import word_tokenize 
from nltk.tokenize import WordPunctTokenizer
import pandas as pd
import os
import glob
import shutil
import os.path
import numpy as np
from pathlib import Path
from tqdm import tqdm
import sentencepiece as spm
from pathlib import Path

#### Read JSON

In [2]:
def extractJson(json_file_path,converted_txt_path): # reading json
    with open(json_file_path, 'r') as jsonfile:
        json_data = json.loads(jsonfile.read())
        paragraphs = json_data['paragraphs']

    converted_txt = open(converted_txt_path,'w')

    for para in paragraphs:
        mainPara = ""
        for item in para['para']:
            mainPara += item
        converted_txt.write(str(mainPara))
            
        converted_txt.write('\n')
        subPara = para['subPara']
        
        for k,v in subPara.items():
            converted_txt.write(str(v))
            
        converted_txt.write('\n')
        blockQ = ""
        
        for i in para['blockQuotes']:
            blockQ += i
            
        converted_txt.write(str(blockQ))

        converted_txt.write('\n')
    converted_txt.close()
    jsonfile.close()

#### Sentence Tokenizer

In [10]:
def sentence_tokenize(document): 
    
    model = io.BytesIO()

    sp = spm.SentencePieceProcessor(model_proto=model.getvalue())

    trainer = PunktTrainer()

    mangle = functools.partial(re.sub, r'([MD]rs?[.]) ([A-Z])', r'\1_\2')

    sent_tokenize = nltk.data.load('tokenizers/punkt/english.pickle')

#     trainer.train(document)
    sample = mangle(document)

    tokenizer = PunktSentenceTokenizer(trainer.get_params())
    excp_list = ["anr", "supp","art","p","no","v","ltd","supra","vs","co","mr","dr","ex","s","sec","section","sections","act","art","article","eg","mohd","etc","suppl","vs","(","[","rs","asst","smt"]
    tokenizer._params.abbrev_types.update(excp_list)
    tokenize_sentence = []
        
    for a_sent in tokenizer.tokenize(document):
        string_split = word_tokenize(a_sent)
        word_count = len(string_split)
        
        if  word_count > 110:
            
            toksList = sp.encode(a_sent,out_type=str) #

            storeIndex = []
            count = 0 #to store the index of '▁' occuring
            
            for i in toksList:
                if i == '▁':
                    storeIndex.append([i,count])
                count = count + 1
                
            half = int(len(toksList)/2) #storing the index of mid length of all the '▁' indices

            regex = re.compile('[,()}{:]')
            
            if(len(storeIndex) != 0):
                for ind in storeIndex: #looping through all occurences of '▁' indices in the main toksList
                    lstInd = ind[1] #index of the '▁' since ind is of the form: ['▁', 6]

                    if(lstInd+1<len(toksList)):
                        
                        if((half <= lstInd) and (is_number(toksList[lstInd+1]) == False) and (regex.search(toksList[lstInd+1]) == None)): #first condition -> to make sure to split only after at least half the length. second condition -> to check if the next token is a number or not, if yes, do not split there. third condition -> to check if any brackets and anything else that is considered to be part of an entity is not split up.
                            
                            #performing detokenization
                            x1 = "".join(toksList[:lstInd]) #adding all the tokens as a string till '▁' is encountered
                            y1 = x1.replace('▁', ' ') #first part of the sentence

                            tokenize_sentence.append(y1)
                            
                            #performing detokenization
                            x2 = "".join(toksList[lstInd+1:]) #adding all the tokens as a string after '▁' is encountered
                            y2 = x2.replace('▁', ' ') #second part of the sentence

                            tokenize_sentence.append(y2)
                            break #since we only want two parts
      
            
        else:
            tokenize_sentence.append(a_sent)
       
    return tokenize_sentence

####  Adding Sentence Separator(@@@@@)

In [4]:
# adding sentence separator 
def addSplitter(raw_txt, final_txt_path):
    final_txt = open(final_txt_path,'w')
    with open(raw_txt,'r') as txt_file:
        while True:
            line = txt_file.readline()
             
            if not line: # if line is empty
                break

            if(line != "\n"):
                sent_split_list = sentence_tokenize(line)
                for sent in sent_split_list:
                    final_txt.write(sent)
                    final_txt.write("@@@@@")
            else:
                final_txt.write(line)
    final_txt.close()

In [5]:
def callForExtraction(json_path, converted_txt_path, final_txt_path): # extracting data from json to text file
    extractJson(json_path,converted_txt_path)
    addSplitter(converted_txt_path, final_txt_path) # after this, you will have a new file with sentence separators 

#### Final code snippet to be called for export

In [15]:
parent_path = Path(r'/Users/arya/Desktop/parentJSON') # JSON files' path
target_path = Path(r'/Users/arya/Desktop/targetText') # brat input data(txt files) path
for a_file in parent_path.rglob('*.json'):
    converted_txt_path = Path(r'/Users/arya/Desktop/parentJSON/temp.txt') # change a path for temp file
    json_path = a_file
    a_file_name = a_file.name.replace(".json",".txt").replace(" ","")    
    final_txt_path = os.path.join(target_path, a_file_name)
    ann_file_name = a_file.name.replace(".json",".ann").replace(" ","")
    ann_file_path =os.path.join(target_path, ann_file_name)
    callForExtraction(json_path, converted_txt_path, final_txt_path)
    open(ann_file_path, 'w').close()
print("Data Extracted")

Data Extracted
